# Capstone Project - The Battle of the Neighborhoods (Week 2)

### Applied Data Science Capstone by IBM/Coursera

#### Name:  Soichiro Tanabe

## Table of contents

1. Introduction: Business Problem

2. Data

3. Methodology　

4. Analysis　

5. Results and Discussion　

6. Conclusion　

## 1. Introduction: Business Problem

Nowadays, the Japanese government has identified the culture industry as one of five potential areas of growth and established a new Creative Industries Promotion Office to promote cultural and creative industries. And this office has introduced Japanese culture such as Japanese pop culture, animation,arts and Japanese foods.These Japanese cultural creative industries are called **"Cool Japan"** (https://en.wikipedia.org/wiki/Cool_Japan) .

Through this promotion, Japanese shops or stores have been founded all over the world. For example, there is a 'Japan House' (https://www.japanhouse.jp/) which is the center to introduce Japanese arts, pop culture and traditional Japanese foods. This Japan House is located in Los Angeles, Sao Paulo and London.

And then, the question of this project is  **'to promote the project of "Cool Japan" and introduce Japanese culture or food, where can we establish an appropriate place  for Japanese center in the USA?'**

An ideal location has to be the place where the people who are interested in cultual things gather such as a place **near parks or theaters** ,as the Japanese center is  cultural. And then, the place where **restaurants, cafes and food malls** are located are preferred, because Japanese food will be also introduced in the Japanese center.

## 2. Data

### 2.1 Data sources

List of all the cities in United States with population density and coordinates:

 https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population  (This sorce is used for obtaining each city in the USA with population)

Following data sources will be needed to extract/generate the required information:

#### ・Google Maps API geocoding

Centers of candidate areas will be generated algorithmically with approximate addresses of of centers of those areas by the use of Goodle Maps API geocoding

#### ・ Foursquare API

The number of all venues in each city and types in every neighborhood such as restauransts or shops will be obtained by using Foursquare API

Using the above data the best city will be selected best to proceed with based on the values like Population density, number of venues through giving weights to each venue based on its category.

### 2.2 Data Cleaning 

#### Importing all the necessary libraries we will be needing to do the Ananlysis ##

In [28]:
# Importing all the necessary libraries we will be needing to do the Ananlysis


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Extracting the content in a wiki page that has 'List of US Cities by population' into a text file ##

In [29]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)

#### Finding the table that has the data list of all cities with their population, Square Area, Location (coordinates) ##

In [30]:
table = soup.find_all('table')[4]

#### Extracting the table from the webpage into a data frame by specifying the column names ##

In [31]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "del1", "del2", "del3", "Sq.Area", "del5", "population density in Sq Mi", "Population density in Km2", "Location"])
df.head()

,Rank,City,State,del1,del2,del3,Sq.Area,del5,population density in Sq Mi,Population density in Km2,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


 #### Finding the radius of each city with the help of Sq.Area, this step involves in preprocessing of the the column Sq.Area (changing its data type to float) and find its square root ###

In [32]:
new= df["Sq.Area"].str.split("s", n=1, expand = True)
new = new[0].str.replace(u'\xa0',u'')
df["Sq.Area"] = new.str.replace(',','')
df["Sq.Area"] = df["Sq.Area"].astype(float)
df["Radius"] = np.sqrt(df["Sq.Area"])

#### Drop unnecessary columns like Rank, del1, del2 ... del5, population density in Sq Mi from the data frame we extracted from the webpage ###

In [33]:
df.drop(columns = ["Rank", "del1", "del2", "del3", "del5", "Sq.Area", "population density in Sq Mi"], inplace = True)
df.head()

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824


#### Splitting the cooridnates to Latitudes and Longitudes for each city ###

In [34]:
df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
df.head()

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


In [35]:

new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)

In [36]:
Latitude = []
Longitude = []
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3]) 

k["Latitude"] = Latitude
k["Longitude"] = Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns = ["Location"], inplace = True)
k.head()
df = k.copy(deep = True)

In [37]:
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head()

,City,State,Population density in Km2,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901


#### Plotting all the cities of the USA from Wiki page, using their coordinates ##

In [38]:
# create map of USA cities that we have using latitude and longitude values
map_tohood = folium.Map(location=[37.0902,-95.7129], zoom_start=3)

# add markers to map
for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

#### Definition of Four Square Credentials and version ##

In [1]:
CLIENT_ID = 'GUH34B53QV3DL0RD4H0FN0F3AOVSO3DJOI1KUG3LKSAAEJKM' # my Foursquare ID     
CLIENT_SECRET = '10WCVJOUTBYHINEKGAQLGRWVM1ZUGV3AY2JXLXSOEFGBEB4L' # my Foursquare Secret 
VERSION = '20180604'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GUH34B53QV3DL0RD4H0FN0F3AOVSO3DJOI1KUG3LKSAAEJKM
CLIENT_SECRET:10WCVJOUTBYHINEKGAQLGRWVM1ZUGV3AY2JXLXSOEFGBEB4L


#### Adding the system of  Foursquare API working into the dataframe which is extracted necessary columns and then, creating the new colums, which shows venues and their categories

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
df_venues = getNearbyVenues(names = df['City'], latitudes = df['Latitude'],longitudes = df['Longitude'], radius = df['Radius'])
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York[d],40.6635,-73.9387,Super Power,40.673952,-73.950184,Tiki Bar
1,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden
2,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
3,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum
4,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater


## 3. Methodology

In this project we will direct our efforts on filtering  areas of cities in the USA that is near parks or theaters, restaurants and cafes. We will give weights to each venue according to kinds of catefory. 

In the first step,  we have collected the required data: location and type (category) of every venue  every city in the USA. We have also assign weights (according to Foursquare categorization).

The Second step in our analysis will identify the ideal city through the data which was added weights into.   

In the third and final step, we will focus on most promising area and assign more precious wights to each categotycreate. And then we will  create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## 4. Analysis

#### Assigning weights to some important categories  (as these are subsets of ideal  situation for the Japanese center) ###

In [56]:
k = df_venues.copy(deep = True)
weights_dict={'Movie Theater':4,'Park':3,'Restaurant':5,'Concert Hall':2.5,'Café':4,'Coffee Shop':3.5,'Food Court':4,'Nightclub':4,'Toy / Game Store':3,'Theme Park Ride / Attraction':4,'Pub':3}
data = df_venues['Venue Category']
allVenues = data.astype(str)

In [57]:
weights = []
for i in allVenues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
df_venues['weights'] = weights;
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,New York[d],40.6635,-73.9387,Super Power,40.673952,-73.950184,Tiki Bar,0.0
1,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden,0.0
2,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0.0
3,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum,0.0
4,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0.0


#### Dropping the rows that we are not giving any weight

In [58]:
df_venues.drop(df_venues[df_venues.weights < 1.0].index, inplace=True)
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
7,New York[d],40.6635,-73.9387,Prospect Park,40.661971,-73.971226,Park,3.0
21,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.027115,-118.387637,Coffee Shop,3.5
28,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.059310,-118.419797,Coffee Shop,3.5
29,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,33.980027,-118.408020,Coffee Shop,3.5
35,Los Angeles,34.0194,-118.4108,Urth Caffé,34.062620,-118.399206,Café,4.0


#### Copying only the relevant columns of cities and weights to group all the venues by city and calculating the means for each city ###

In [59]:
citywise_venues_weights = df_venues[['City','weights']].copy()
citywise_venues_weights_means = citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means = citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means.head()

,City,weights
0,Abilene,3.5
1,Akron,4.0
2,Albuquerque,4.0
3,Alexandria[m],3.5
4,Allen,3.5


#### Merging the table for which we calculated the means of weights to the actual table  from the wiki page.###

In [61]:
city_selection = pd.merge(df, citywise_venues_weights_means, on='City')
city_selection = city_selection[['City','Population density in Km2','weights']].copy()
city_selection.head()

,City,Population density in Km2,weights
0,New York[d],"10,933/km2",3.000000
1,Los Angeles,"3,276/km2",3.700000
2,Chicago,"4,600/km2",3.250000
3,Houston[3],"1,395/km2",3.166667
4,Phoenix,"1,200/km2",3.500000


####  Preprocessing the population density in Km2 column as we have to normalize these values

In [63]:

k = city_selection.copy(deep = True)
k['Population density in Km2'] = k['Population density in Km2'].str.split("/", n = 0, expand = True)
k['Population density in Km2'] = k['Population density in Km2'].str.replace(',','')
k['Population density in Km2'] = k['Population density in Km2'].astype(float)
city_selection = k.copy(deep = True)
city_selection.head()

,City,Population density in Km2,weights
0,New York[d],10933.0,3.000000
1,Los Angeles,3276.0,3.700000
2,Chicago,4600.0,3.250000
3,Houston[3],1395.0,3.166667
4,Phoenix,1200.0,3.500000


#### Normalizing our dataframe ###

In [64]:

from sklearn import preprocessing
column_names_to_normalize = ['Population density in Km2', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head()

,City,Population density in Km2,weights
0,New York[d],1.000000,0.200000
1,Los Angeles,0.295260,0.480000
2,Chicago,0.417119,0.300000
3,Houston[3],0.122135,0.266667
4,Phoenix,0.104188,0.400000


#### Calculating the sum of normalized each column('Population denstity in km2' and 'weights') to determine the city of the maximum sum and conclude the most suitable locality in that city ###

In [65]:

city_selection['sum'] = city_selection['Population density in Km2'] + city_selection['weights']
row_num = city_selection['sum'].argmax()
city_name = city_selection['City'].iloc[row_num]
city_name

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()


'Cambridge'

####  Finding the state in which the city belongs

In [66]:

row = df.loc[df['City']== city_name].index[0]
state_name = df['State'].iloc[row]
state_name

'Massachusetts'

 #### As we have concluded the  location in Cambridge, Massachusetts,  it is considered as one of the best places to procced for setting up a Japanese Center ###

#### Let us now check which location in Massachusetts would be the best to start a Japaense Center ###

In [67]:

lat_Massachusetts = df['Latitude'].iloc[row]
long_Massachusetts = df['Longitude'].iloc[row]
print(lat_Massachusetts, long_Massachusetts)

42.376 -71.1187


 #### Getting all the venues of Massachusetts using Foursquare API within our considered radius and limits ###

In [70]:
def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   # print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


Massachusetts_venues = getNearbyVenues1(name = 'Massachusetts', latitudes = lat_Massachusetts ,longitudes = long_Massachusetts, radius = 2500)
Massachusetts_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Massachusetts,38.3539,-121.9728,Harvard Square,42.373458,-71.119004,Plaza
1,Massachusetts,38.3539,-121.9728,Felipe's Mexican Taqueria,42.373431,-71.120515,Mexican Restaurant
2,Massachusetts,38.3539,-121.9728,Sanders Theatre,42.375835,-71.114786,Theater
3,Massachusetts,38.3539,-121.9728,Club Passim,42.374330,-71.120083,Music Venue
4,Massachusetts,38.3539,-121.9728,The Sinclair,42.374094,-71.120757,Rock Club


In [71]:
venues_in_Massachusetts = Massachusetts_venues.copy(deep = True)
venues_in_Massachusetts.shape

(100, 7)

 #### As we have got more categories, we are now giving weights for each category for better results ###

In [73]:


k = Massachusetts_venues.copy(deep = True)
new_weightage_dict= {'Coffee Shop' : 3, 
'Caribbean Restaurant':3,
'Distillery':2,
'Fish & Chips Shop':3,
'Science Museum':3,
'Latin American Restaurant':3,
'Restaurant':5,
'State / Provincial Park':4,
'Diner':1,
'Supermarket':1,
'Bar':1,
'Jazz Club':1,
'Golf Course':2,
'Park':4,
'Cajun / Creole Restaurant':2,
'Bakery':2,
'Go Kart Track':2,
'Taco Place':3,
'Hot Dog Joint':2,
'Food Truck':3,
'Beer Garden':3,
'Boutique':4,
'Café':5,
'Bagel Shop':1,
'Record Shop':3,
'Bakery':1,
'Pizza Place':1,
'Ramen Restaurant':4,
'Wine Bar':3,
'Middle Eastern Restaurant':2,
'French Restaurant':2,
'Theater':3,
'Lounge':3,
'Wine Shop':3,
'Cocktail Bar':2,
'New American Restaurant':3,
'Residential Building (Apartment / Condo)':3,
'Pool':2,
'Burger Joint':3,
'Cheese Shop':2,
'Coffee Shop':3,
'Bagel Shop':1,
'Vietnamese Restaurant':1,
'Portuguese Restaurant':1,
'Ice Cream Shop':3,
'Italian Restaurant':3,
'Gym':1,
'Farmers Market':2,
'Bar':3,
'Pizza Place':3,
'Bakery':2,
'Bookstore':3,
'Bar':3,
'Farmers Market':4,
'Asian Restaurant':5,
'Tea Room':1,
'Donut Shop':1,
'Historic Site':1,
'Gym / Fitness Center':1,
'Mexican Restaurant':3,
'Plaza':2,
'Gay Bar':2,
'Bar':3,
'College Administrative Building':3,
'Mexican Restaurant':2,
'Bakery':3,
'American Restaurant':3,
'Café':4,
'New American Restaurant':3,
'Chocolate Shop':1,
'Gym':1,
'Grocery Store':1,
'Middle Eastern Restaurant':3,
'American Restaurant':2,
'Frozen Yogurt Shop':2,
'Japanese Restaurant':5,
'Bar':3,
'Liquor Store':3,
'Ice Cream Shop':2,
'Fish Market':3,
'Indie Movie Theater':3,
'Grocery Store':4,
'Modern European Restaurant':5,
'American Restaurant':1,
'Poke Place':1,
'Ramen Restaurant':1,
'Diner':1,
'Brewery':1,,
'Fried Chicken Joint':2,
'Beer Garden':3,
'Gym / Fitness Center':2,
'Vietnamese Restaurant':3,
'Italian Restaurant':3,
'Pet Store':3}

#### Plotting all the venues that we have got from the Foursquare API ###

In [74]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_Massachusetts, long_Massachusetts], zoom_start=15) # generate map centred around Jersey city


# add Massachusetts as a red circle mark
folium.features.CircleMarker(
    [lat_Massachusetts, long_Massachusetts],
    radius=10,
    popup='Massachusetts',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

In [75]:

# add all the venuew of Cambridge, Massachusetts city to the map as blue circle markers
for lat, lng, label in zip(venues_in_Massachusetts['Venue Latitude'], venues_in_Massachusetts['Venue Longitude'], venues_in_Massachusetts['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map


 #### Calculating new weights for our dataframe with all categories

In [76]:
# Calculating new weights for our data frame as we have given weights for all categories

allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,Massachusetts,38.3539,-121.9728,Harvard Square,42.373458,-71.119004,Plaza,2
1,Massachusetts,38.3539,-121.9728,Felipe's Mexican Taqueria,42.373431,-71.120515,Mexican Restaurant,2
2,Massachusetts,38.3539,-121.9728,Sanders Theatre,42.375835,-71.114786,Theater,3
3,Massachusetts,38.3539,-121.9728,Club Passim,42.374330,-71.120083,Music Venue,0
4,Massachusetts,38.3539,-121.9728,The Sinclair,42.374094,-71.120757,Rock Club,0


In [77]:
# Dropping unnecessary columns 

newframe = k[['City','Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe

,Venue Latitude,Venue Longitude,weights
Venue Category,,,
American Restaurant,42.376823,-71.120851,1
Arepa Restaurant,42.371933,-71.120612,0
Art Museum,42.374206,-71.114368,0
Arts & Crafts Store,42.386663,-71.119256,0
Bagel Shop,42.386047,-71.119370,1
Bakery,42.380732,-71.119239,3
Bar,42.385130,-71.119240,3
Bookstore,42.377002,-71.119137,3
Brewery,42.381988,-71.106219,1


#### Using K Means algorithm to cluster the venues and calculating the weights for each cluster to decide which cluster would be the best area to locate a Japanese Center #

In [78]:
# Cluster them using K means algorithm 
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Venue Category,weights,Venue Latitude,Venue Longitude,clusters
0,American Restaurant,1,42.376823,-71.120851,1
1,Arepa Restaurant,0,42.371933,-71.120612,1
2,Art Museum,0,42.374206,-71.114368,1
3,Arts & Crafts Store,0,42.386663,-71.119256,0
4,Bagel Shop,1,42.386047,-71.119370,0


#### New group by clusters and add weights of each cluster 

In [79]:

finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,0.666667,42.381394,-71.112194
1,0.200000,42.372627,-71.117818
2,3.000000,42.376551,-71.118922


#### Final coordinates of the place where we will be setting up a Japanese center is the one that has maximum weight for, in the above data frame

In [81]:
lat1 = 42.376551
long1 = -71.118922

#### As we have a location with the maximum weight, let us plot the same in the graph with a circle of radius 50M so that we can locate a Japanese center with in that premices ##

In [82]:
# create map of the venues that we have using latitude and longitudes
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Massachusetts


# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='Japanese Center can be installed within this circle',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map

## Finally, we found that the appropriate place is along Cambridge Street in Cambridge, Massachusetts. 

### I found that the appropriate place is along Cambridge Street in Cambridge, Massachusetts. We can find that this area is surrounded by university facilities. On the south side of this location, many restaurants and cafes are located. There are also huge parks in front of the venue and around this location. Sandra Theater is found on the west side of this location. As this is a cultural place where many people gather, it is considered that this satisfies our requirements.

## Further enhancements and drawbacks in continuing this approach:
### This project can be enhanced by considering many more attributes to define the weights and analyze and also by extending the LIMIT and Radius of the search. As the API limit in the free trail of four square API was used, the research was regulated within a small Radius.
